In [8]:
# # Do not forget to check if your notebook log collection plugin is working
from mining_extension import check_logging
check_logging("http://3.249.245.244:9999")

Request successful!


In [32]:
import catboost
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import sklearn
import numpy as np

In [10]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [11]:
train_features

,filename,cell_type,cell_number,execution_count,linesofcomment,linesofcode,variable_count,function_count,text/plain,image/png,...,helper_functions,load_data,data_exploration,data_preprocessing,evaluation,modelling,prediction,result_visualization,save_results,comment_only
0,nb_54880.ipynb,code,0,1,0.0,2.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,nb_54880.ipynb,code,1,2,1.0,4.0,4.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,nb_54880.ipynb,code,2,3,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
3,nb_54880.ipynb,code,3,4,0.0,1.0,0.0,0.0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,nb_54880.ipynb,code,4,5,0.0,1.0,1.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,nb_95821.ipynb,code,27,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
5829,nb_95821.ipynb,code,29,-1,0.0,2.0,1.0,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
5830,nb_95821.ipynb,code,31,-1,0.0,3.0,2.0,0.0,0,0,...,0,0,1,1,0,0,0,0,0,0
5831,nb_95821.ipynb,code,33,-1,5.0,17.0,14.0,0.0,0,0,...,0,0,0,0,1,1,1,0,0,0


In [20]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5833 entries, 0 to 5832
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   filename              5833 non-null   object 
 1   cell_type             5833 non-null   object 
 2   cell_number           5833 non-null   int64  
 3   execution_count       5833 non-null   int64  
 4   linesofcomment        5833 non-null   float64
 5   linesofcode           5833 non-null   float64
 6   variable_count        5833 non-null   float64
 7   function_count        5833 non-null   float64
 8   text/plain            5833 non-null   int64  
 9   image/png             5833 non-null   int64  
 10  text/html             5833 non-null   int64  
 11  execute_result        5833 non-null   int64  
 12  display_data          5833 non-null   int64  
 13  stream                5833 non-null   int64  
 14  error                 5833 non-null   int64  
 15  text                 

In [41]:
train_features.dtypes[3]

dtype('int64')

According to columns description I made a decision to divide all the columns by type

In [48]:
cat_cols = ['cell_type']

In [49]:
one_hot_cols = ['error','stream','display_data', 'text/plain', 'image/png', 'text/html', 'execute_result']

In [50]:
num_cols = ['cell_number', 'execution_count', 'linesofcomment', 'linesofcode', 'variable_count']

In [51]:
text_cols = ['comment', 'filename', 'text', 'code_line_before', 'code_line_after', 'markdown_heading', 'packages_info']

In [53]:
train_features = train_features.drop(text_cols, axis = 1)

In [54]:
train_labels = train_features['primary_label']
train_labels

0         helper_functions
1                load_data
2         data_exploration
3         data_exploration
4       data_preprocessing
               ...        
5828            evaluation
5829             modelling
5830    data_preprocessing
5831             modelling
5832            evaluation
Name: primary_label, Length: 5833, dtype: object

In [ ]:
train

In [55]:
cbc = CatBoostClassifier(eval_metric='F1', boosting_type = 'Plain',loss_function='MultiClass', score_function='Cosine')

In [59]:
train_d = train_features.drop(['primary_label'], axis = 1)
# test_d = test_features.drop(['primary_label'], axis = 1)
validation_d = validation_features.drop(['primary_label'], axis = 1)

train_labels = train_labels
# test_labels = test_features['primary_label']
validation_labels = validation_features['primary_label']


from catboost import Pool

train_data = Pool(train_d,
    label = train_labels,
    cat_features=cat_cols+one_hot_cols
)

eval_data = Pool(validation_d,
    label = validation_labels,
    cat_features=cat_cols+one_hot_cols
)


grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]
       }

grid_search_result = cbc.grid_search(grid, 
                                       X=train_data, 
                                       y=train_labels, 
                                       plot=True)



CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="nb_128972.ipynb": Cannot convert 'b'nb_128972.ipynb'' to float

In [ ]:
from catboost import CatBoostClassifier, Pool

train_data = Pool(,
    label = [1, 0, 0, 1],
    cat_features=[3],
    embedding_features=[0, 1]
)

eval_data = Pool(
    [
        [[0.2, 0.1, 0.3], [1.2, 0.3], 1, "female"],
        [[0.33, 0.22, 0.4], [0.98, 0.5], 2, "female"],
        [[0.78, 0.29, 0.67], [0.76, 0.34], 2, "male"],
    ],
    label = [0, 1, 1],
    cat_features=[3],
    embedding_features=[0, 1]
)

model = CatBoostClassifier(iterations=10)

model.fit(train_data, eval_set=eval_data)
preds_class = model.predict(eval_data)
